In [182]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=6, app_name='west0', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 6 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [196]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [186]:
device = "pc"

#16.1 update  
PC: 2/16 ~ 3/1  
CONSOLE: 2/24 ~ 3/9

In [183]:
# Day 14
oct_start_date = "2021-10-06"
oct_end_date = "2021-10-19"
nov_start_date = "2021-11-03"
nov_end_date = "2021-11-16"
dec_start_date = "2021-12-01"
dec_end_date = "2021-12-14"

feb_start_date = "2022-02-16"
feb_end_date = "2022-03-01"
march_start_date = "2022-03-16"
march_end_date = "2022-03-29"

In [187]:
# Day 28
oct_start_date = "2021-10-06"
oct_end_date = "2021-11-02"
nov_start_date = "2021-11-03"
nov_end_date = "2021-11-30"
dec_start_date = "2021-12-01"
dec_end_date = "2021-12-28"

feb_start_date = "2022-02-16"
feb_end_date = "2022-03-15"
march_start_date = "2022-03-16"
march_end_date = "2022-04-12"

In [188]:
gcoin10 = load_data_mart(device, oct_start_date, oct_end_date, "gcoin_use")
gcoin11 = load_data_mart(device, nov_start_date, nov_end_date, "gcoin_use")
gcoin12 = load_data_mart(device, dec_start_date, dec_end_date, "gcoin_use")
gcoin2 = load_data_mart(device, feb_start_date, feb_end_date, "gcoin_use")
gcoin3 = load_data_mart(device, march_start_date, march_end_date, "gcoin_use")

no data exists in
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2022-04-11
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2022-04-09
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2022-04-08
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2022-04-12
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2022-04-10


### WSUS

In [189]:
gcoin10_wsus = gcoin10.where(col("event_type") == "wsus")
gcoin11_wsus = gcoin11.where(col("event_type") == "wsus")
gcoin12_wsus = gcoin12.where(col("event_type") == "wsus")
gcoin2_wsus = gcoin2.where(col("event_type") == "wsus")
gcoin3_wsus = gcoin3.where(col("event_type") == "wsus")

In [190]:
gcoin = gcoin10_wsus.unionByName(gcoin11_wsus)
gcoin = gcoin.unionByName(gcoin12_wsus)
gcoin = gcoin.unionByName(gcoin2_wsus)
gcoin = gcoin.unionByName(gcoin3_wsus)

In [191]:
gcoin.where(col("paid_use") > 0).select(sum("paid_use")/sum(col("paid_use") + col("free_use"))).show()

+--------------------------------------------+
|(sum(paid_use) / sum((paid_use + free_use)))|
+--------------------------------------------+
|                          0.8960136760251033|
+--------------------------------------------+



In [112]:
tmp10 = gcoin10_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("october"))
tmp11 = gcoin11_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("november"))
tmp12 = gcoin12_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("december"))
tmp2 = gcoin2_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("feburary"))
tmp3 = gcoin3_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("march"))

In [113]:
wsus_by_month = tmp10.unionByName(tmp11)
wsus_by_month = wsus_by_month.unionByName(tmp12)
wsus_by_month = wsus_by_month.unionByName(tmp2)
wsus_by_month = wsus_by_month.unionByName(tmp3)

In [116]:
wsus_by_month_df = wsus_by_month.toPandas()

In [118]:
# wsus_by_month_df.to_csv("../Data/wsus_by_month_df.csv", index=False)

In [119]:
wsus_by_month_df.groupby(["price", "buy_cnt_group"])["user_cnt"].describe()

count      mean            std      min       25%  \
price buy_cnt_group                                                      
200   None             5.0  262864.4  170496.339029  46343.0  199026.0   
      multiple         5.0   57256.2   16966.699847  39984.0   41797.0   
      one              5.0   47031.2   16975.571616  22290.0   43668.0   
1800  None             5.0   38497.2   38804.072231   1724.0   18977.0   
      multiple         5.0   58780.2   11992.424930  40736.0   55031.0   
      one              5.0   30145.8    8529.067927  19438.0   28769.0   

                          50%       75%       max  
price buy_cnt_group                                
200   None           230079.0  332218.0  506656.0  
      multiple        55115.0   73819.0   75566.0  
      one             44641.0   56867.0   67690.0  
1800  None            20934.0   50387.0  100464.0  
      multiple        61121.0   63982.0   73031.0  
      one             28980.0   30241.0   43301.0

In [192]:
wsus10 = gcoin10_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("october"))
wsus11 = gcoin11_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("november"))
wsus12 = gcoin12_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("december"))
wsus2 = gcoin2_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("feburary"))
wsus3 = gcoin3_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("march"))

In [193]:
wsus_account = wsus10.unionByName(wsus11)
wsus_account = wsus_account.unionByName(wsus12)
wsus_account = wsus_account.unionByName(wsus2)
wsus_account = wsus_account.unionByName(wsus3)

In [161]:
wsus_account.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- price: long (nullable = true)
 |-- buy_cnt: long (nullable = true)
 |-- buy_cnt_group: string (nullable = false)
 |-- month: string (nullable = false)



In [194]:
# Day 28, 3월까지 다 했을때 (4월 몇일 빠져있는 상황)
wsus_account.groupBy("month", "price", "buy_cnt_group").agg(sum("buy_cnt").alias("total_buy_cnt")).toPandas().groupby(["price", "buy_cnt_group"])["total_buy_cnt"].describe()

count      mean            std       min       25%  \
price buy_cnt_group                                                       
200   None             0.0       NaN            NaN       NaN       NaN   
      multiple         5.0  528119.0  140128.550703  387310.0  411408.0   
      one              5.0   66400.6   17959.056178   41470.0   57702.0   
1800  None             0.0       NaN            NaN       NaN       NaN   
      multiple         5.0  701847.0  168807.233567  440931.0  638488.0   
      one              5.0   41333.0   11121.326877   32169.0   36633.0   

                          50%       75%       max  
price buy_cnt_group                                
200   None                NaN       NaN       NaN  
      multiple       495616.0  641648.0  704613.0  
      one             65654.0   82721.0   84456.0  
1800  None                NaN       NaN       NaN  
      multiple       744249.0  829221.0  856346.0  
      one             36659.0   40728.0   60476.0

In [195]:
# Day 28, 2월까지
wsus_account = wsus10.unionByName(wsus11)
wsus_account = wsus_account.unionByName(wsus12)
wsus_account = wsus_account.unionByName(wsus2)
wsus_account.groupBy("month", "price", "buy_cnt_group").agg(sum("buy_cnt").alias("total_buy_cnt")).toPandas().groupby(["price", "buy_cnt_group"])["total_buy_cnt"].describe()

count       mean            std       min        25%  \
price buy_cnt_group                                                         
200   None             0.0        NaN            NaN       NaN        NaN   
      multiple         4.0  557296.75  143198.255328  387310.0  468539.50   
      one              4.0   66587.25   20731.731161   41470.0   53644.00   
1800  None             0.0        NaN            NaN       NaN        NaN   
      multiple         4.0  767076.00   98127.603317  638488.0  717808.75   
      one              4.0   41484.25   12835.862713   32169.0   35517.00   

                          50%        75%       max  
price buy_cnt_group                                 
200   None                NaN        NaN       NaN  
      multiple       568632.0  657389.25  704613.0  
      one             70211.5   83154.75   84456.0  
1800  None                NaN        NaN       NaN  
      multiple       786735.0  836002.25  856346.0  
      one             36646.0   42613.25   60476.0

### Buy Count by Price Monthly

In [130]:
tmp10_price = gcoin10.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("october"))
tmp11_price = gcoin11.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("november"))
tmp12_price = gcoin12.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("december"))
tmp2_price = gcoin2.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("feburary"))
tmp3_price = gcoin3.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("march"))

In [131]:
price_by_month = tmp10_price.unionByName(tmp11_price)
price_by_month = price_by_month.unionByName(tmp12_price)
price_by_month = price_by_month.unionByName(tmp2_price)
price_by_month = price_by_month.unionByName(tmp3_price)

In [132]:
price_by_month_df = price_by_month.toPandas()
price_by_month_df.to_csv("../Data/price_by_month_df.csv", index=False)

In [144]:
price_by_month_df[(price_by_month_df.price >= 100) & (price_by_month_df.price <= 250)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,100,16838.000000
1,120,357.500000
2,140,629.333333
3,150,2652.400000
4,170,256.666667
5,180,106.000000
6,200,403609.800000
7,210,544.666667
8,240,403.000000
9,250,7561.400000


In [179]:
price_by_month_df[(price_by_month_df.price >= 900)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,900,181.800000
1,910,985.000000
2,920,1.000000
3,940,260.333333
4,950,170.333333
5,960,127.333333
6,970,68.666667
7,980,24.000000
8,990,97616.600000
9,1000,10165.600000


In [170]:
price_by_month_df[(price_by_month_df.price >= 1800) & (price_by_month_df.price <= 2700)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,1800,526166.400000
1,1860,8.333333
2,1980,2410.800000
3,1990,1142.000000
4,2000,3513.666667
5,2070,115.000000
6,2080,38.000000
7,2370,66.666667
8,2420,19.000000
9,2490,19647.000000


In [14]:
feb_membership = load_membership("pubg", "pc", "2022-02-16", "2022-03-01").select("date", "account_id", "membership").distinct()

In [15]:
feb_gcoin_pc = feb_gcoin_pc.join(feb_membership, ["date", "account_id"], "left")

In [29]:
feb_gcoin_pc_tmp = feb_gcoin_pc.groupBy("date", "product_id", "product_name", "sales_id", "price", "membership").agg(countDistinct("account_id").alias("pu"), \
                                                                                       countDistinct(when(col("paid_use")>0, col("account_id")).otherwise(0)).alias("paid_pu"), \
                                                                                       sum("free_use").alias("free_use"), \
                                                                                       sum("paid_use").alias("paid_use")
                                                                                      )

In [32]:
mysql.insert_table(feb_gcoin_pc_tmp, "labs", "feb_gcoin_pc")

In [60]:
march_gcoin_pc = load_data_mart("pc", "2022-03-16", "2022-03-29", "gcoin_use")
march_membership = load_membership("pubg", "pc", "2022-03-16", "2022-03-29").select("date", "account_id", "membership").distinct()
march_gcoin_pc = march_gcoin_pc.join(march_membership, ["date", "account_id"], "left")

In [61]:
march_gcoin_pc_tmp = march_gcoin_pc.groupBy("date", "product_id", "product_name", "sales_id", "price", "membership").agg(countDistinct("account_id").alias("pu"), \
                                                                                       countDistinct(when(col("paid_use")>0, col("account_id")).otherwise(0)).alias("paid_pu"), \
                                                                                       sum("free_use").alias("free_use"), \
                                                                                       sum("paid_use").alias("paid_use")
                                                                                      )

In [62]:
mysql.insert_table(march_gcoin_pc_tmp, "labs", "march_gcoin_pc")

In [31]:
# mysql.drop_table("labs", "feb_gcoin_pc")

In [35]:
feb_wsus = feb_gcoin_pc.where(col("event_type") == "wsus")

In [38]:
feb_wsus.select("event_name").distinct().toPandas()

,event_name
0,202201_wsus
1,202111_wsus
2,202112_wsus_progressive
3,202202_wsus_progressive


In [40]:
feb_wsus_cnt_daily = feb_wsus.groupBy("date", "account_id", "membership", "price").agg(count("*").alias("buy_cnt"), sum(when(col("paid_use") > 0, 1).otherwise(0)).alias("paid_buy_cnt"))

In [44]:
# feb_wsus_cnt_daily_df = feb_wsus_cnt_daily.toPandas()
feb_wsus_cnt_daily_df.to_csv("../Data/feb_wsus_cnt_daily_df.csv", index=False)

In [45]:
feb_wsus_cnt_daily_df.head()

,date,account_id,membership,price,buy_cnt,paid_buy_cnt
0,2022-02-16,account.00ce68eee960409a8deaf46178fc94a5,legacy,200,3,0
1,2022-02-16,account.014d901691864512be0fe258c1ec38ef,legacy,200,1,0
2,2022-02-16,account.02277864522d4e7ead16316efdc90d58,legacy,200,6,0
3,2022-02-16,account.022c73a0054f44b5800372a994969b58,basic,200,2,0
4,2022-02-16,account.0247299a07614ed89f0073fd64e7964d,legacy,200,1,1


In [49]:
feb_wsus_cnt_daily_df.groupby(["date", "membership", "price"]).describe().reset_index().to_csv("../Data/feb_wsus_daily_describe.csv", index=False)

In [58]:
# 상자 오픈 1회
feb_wsus_cnt_daily_df[(feb_wsus_cnt_daily_df.price == 200) & (feb_wsus_cnt_daily_df.buy_cnt == 1)].groupby("date", as_index=False)[["buy_cnt", "paid_buy_cnt"]].sum()

,date,buy_cnt,paid_buy_cnt
0,2022-02-16,38814,8282
1,2022-02-17,39363,6183
2,2022-02-18,34350,4822
3,2022-02-19,36284,4340
4,2022-02-20,34808,3820
5,2022-02-21,15194,2212
6,2022-02-22,14466,2035
7,2022-02-23,17650,4784
8,2022-02-24,15992,3582
9,2022-02-25,15566,3153


In [54]:
feb_wsus_cnt_daily_df.groupby(["date", "price"]).describe()

buy_cnt                                                 \
                    count      mean       std  min  25%  50%  75%    max   
date       price                                                           
2022-02-16 200    88076.0  2.844475  3.732719  1.0  1.0  2.0  3.0  187.0   
           1800   36037.0  4.468685  8.985432  1.0  1.0  1.0  4.0  141.0   
2022-02-17 200    87500.0  2.702994  3.325971  1.0  1.0  2.0  3.0  303.0   
           1800   27900.0  3.507849  7.338281  1.0  1.0  1.0  3.0  124.0   
2022-02-18 200    77035.0  2.695632  3.036430  1.0  1.0  2.0  3.0  179.0   
           1800   23806.0  3.013862  6.151970  1.0  1.0  1.0  2.0  116.0   
2022-02-19 200    81589.0  2.688990  2.929741  1.0  1.0  2.0  3.0  176.0   
           1800   24091.0  2.646964  5.589806  1.0  1.0  1.0  2.0  122.0   
2022-02-20 200    77378.0  2.640983  2.801054  1.0  1.0  2.0  3.0  176.0   
           1800   21845.0  2.463493  5.277816  1.0  1.0  1.0  2.0  124.0   
2022-02-21 200    35649.0  2.728744  2.874092  1.0  1.0  2.0  3.0   88.0   
           1800   12396.0  2.830187  6.195572  1.0  1.0  1.0  2.0  128.0   
2022-02-22 200    34545.0  2.717180  2.803678  1.0  1.0  2.0  3.0  102.0   
           1800   11486.0  2.778078  6.151358  1.0  1.0  1.0  2.0  120.0   
2022-02-23 200    44213.0  2.655870  2.868570  1.0  1.0  2.0  3.0  144.0   
           1800   13979.0  2.610988  5.317455  1.0  1.0  1.0  2.0  112.0   
2022-02-24 200    38570.0  2.620119  2.686001  1.0  1.0  2.0  3.0   88.0   
           1800   11714.0  2.647345  5.307920  1.0  1.0  1.0  2.0  122.0   
2022-02-25 200    37303.0  2.649385  3.172744  1.0  1.0  2.0  3.0  248.0   
           1800   11868.0  2.770391  6.040838  1.0  1.0  1.0  2.0  111.0   
2022-02-26 200    40808.0  2.630612  2.765014  1.0  1.0  2.0  3.0  134.0   
           1800   12869.0  2.904577  6.511741  1.0  1.0  1.0  2.0  105.0   
2022-02-27 200    38884.0  2.674288  2.852493  1.0  1.0  2.0  3.0  108.0   
           1800   12073.0  3.050443  7.199619  1.0  1.0  1.0  2.0  106.0   
2022-02-28 200    31016.0  2.655307  2.723758  1.0  1.0  2.0  3.0   60.0   
           1800    9954.0  3.047016  7.065211  1.0  1.0  1.0  2.0  122.0   
2022-03-01 200    31766.0  2.622615  2.716652  1.0  1.0  2.0  3.0   95.0   
           1800   10106.0  2.799426  6.136534  1.0  1.0  1.0  2.0  110.0   

                 paid_buy_cnt                                                 
                        count      mean       std  min  25%  50%  75%    max  
date       price                                                              
2022-02-16 200        88076.0  1.045949  3.291019  0.0  0.0  0.0  1.0  181.0  
           1800       36037.0  4.049144  8.678071  0.0  0.0  1.0  4.0  115.0  
2022-02-17 200        87500.0  0.741771  2.749882  0.0  0.0  0.0  0.0  262.0  
           1800       27900.0  3.027778  7.147801  0.0  0.0  1.0  3.0  116.0  
2022-02-18 200        77035.0  0.638100  2.399961  0.0  0.0  0.0  0.0  145.0  
           1800       23806.0  2.513946  6.041566  0.0  0.0  1.0  2.0  108.0  
2022-02-19 200        81589.0  0.549829  2.244566  0.0  0.0  0.0  0.0  151.0  
           1800       24091.0  2.099041  5.526838  0.0  0.0  1.0  2.0  111.0  
2022-02-20 200        77378.0  0.484414  2.082843  0.0  0.0  0.0  0.0  170.0  
           1800       21845.0  1.886290  5.229496  0.0  0.0  0.0  1.0  117.0  
2022-02-21 200        35649.0  0.676793  2.319578  0.0  0.0  0.0  0.0   82.0  
           1800       12396.0  2.305663  6.095825  0.0  0.0  1.0  2.0  119.0  
2022-02-22 200        34545.0  0.634650  2.200717  0.0  0.0  0.0  0.0   91.0  
           1800       11486.0  2.221574  6.044827  0.0  0.0  1.0  2.0  116.0  
2022-02-23 200        44213.0  0.855608  2.423246  0.0  0.0  0.0  1.0  144.0  
           1800       13979.0  2.170255  5.284419  0.0  0.0  1.0  2.0  108.0  
2022-02-24 200        38570.0  0.753383  2.201356  0.0  0.0  0.0  1.0   82.0  
           1800       11714.0  2.177736  5.249288  0.0  0.0  1.0  2.0  111.0  
20

## 3월 BM별 카니발라이즈

In [64]:
march_bm = pd.read_csv("../Data/march_carnival.csv")

In [66]:
march_bm.columns = ["bm", "name", "date", "value"]

In [67]:
march_bm.dtypes

bm        object
name      object
date      object
value    float64
dtype: object

In [69]:
march_sales = march_bm[march_bm.name == "Paid GCoin Revenue"]

In [82]:
march_percent = march_bm[march_bm.name != "Paid GCoin Revenue"]

In [83]:
march_percent["value"] = march_percent["value"] * 100

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
march_percent.groupby(["date", "bm"], as_index=False)["value"].sum().pivot(index="date", columns="bm")

value                                                   
bm              esport ingame_items survivor_pass (+levels_up)       wsus
date                                                                     
2022년 3월 16일  6.749736    51.695374                  23.668358  17.886532
2022년 3월 17일  5.000868    53.182361                  18.853316  22.963455
2022년 3월 18일  4.135963    54.442382                  15.086021  26.335633
2022년 3월 19일  3.640249    55.100241                  13.890562  27.368947
2022년 3월 20일  3.338947    55.908356                  12.411723  28.340974
2022년 3월 21일  2.715737    56.081625                  10.885168  30.317470
2022년 3월 22일  2.520000    56.181885                   9.740076  31.558039
2022년 3월 23일  2.337389    56.650404                   9.085129  31.927078
2022년 3월 24일  2.121052    56.892521                   8.835904  32.150524
2022년 3월 25일  2.170793    56.829551                   8.422381  32.577275
2022년 3월 26일  1.783088    56.363115                   7.571692  34.282105
2022년 3월 27일  1.849191    56.893553                   7.799984  33.457272
2022년 3월 28일  1.543490    56.193804                   6.331690  35.931016
2022년 3월 29일  1.559627    56.305840                   6.271429  35.863105

In [73]:
march_sales.groupby(["date", "bm"], as_index=False)["value"].sum().pivot(index="date", columns="bm")

value                                                   
bm               esport ingame_items survivor_pass (+levels_up)       wsus
date                                                                      
2022년 3월 16일  376047.91   2880103.50                 1318634.84  996512.05
2022년 3월 17일  138954.95   1477733.85                  523861.35  638066.35
2022년 3월 18일   77759.20   1023557.40                  283628.45  495129.55
2022년 3월 19일   56396.00    853632.00                  215197.40  424009.20
2022년 3월 20일   43798.80    733380.65                  162811.40  371764.15
2022년 3월 21일   26938.90    556304.65                  107976.00  300735.75
2022년 3월 22일   22763.70    507503.05                   87984.20  285070.55
2022년 3월 23일   22749.50    551370.85                   88424.35  310742.00
2022년 3월 24일   18560.40    497841.70                   77319.15  281335.25
2022년 3월 25일   20894.10    546990.05                   81066.25  313559.50
2022년 3월 26일   20115.00    635831.85                   85416.20  386736.15
2022년 3월 27일   18111.20    557222.40                   76394.00  327684.60
2022년 3월 28일   14008.20    509996.20                   57464.30  326097.90
2022년 3월 29일   13418.10    484422.00                   53955.65  308544.85

In [75]:
march_sales.groupby(["bm"], as_index=False)["value"].sum()

,bm,value
0,esport,870515.96
1,ingame_items,11815890.15
2,survivor_pass (+levels_up),3220133.54
3,wsus,5765987.85
